## Demo 3: HKR classifier on MNIST dataset
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/deel-ai/deel-lip/blob/master/docs/notebooks/demo3.ipynb)

This notebook will demonstrate learning a binary task on the MNIST0-8 dataset.

In [1]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

In [2]:
# pip install deel-lip -qqq

In [3]:
import keras
import keras.ops as K
from keras.layers import Input, Flatten, Dense
from keras.optimizers import Adam
from keras.metrics import BinaryAccuracy

# from keras.models import Sequential
from deel.lip.model import Sequential

from deel.lip.layers import (
    SpectralDense,
    SpectralConv2D,
    ScaledL2NormPooling2D,
    FrobeniusDense,
)
from deel.lip.activations import GroupSort, GroupSort2
from deel.lip.losses import HKR, KR, HingeMargin, MulticlassHKR, MulticlassKR

import numpy as np

2025-04-24 17:37:54.468846: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745509074.489448   25373 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745509074.495521   25373 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-24 17:37:54.516026: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### data preparation

For this task we will select two classes: 0 and 8. Labels are changed to {-1,1}, wich is compatible
with the Hinge term used in the loss.

In [4]:
from keras.datasets import mnist

# first we select the two classes
selected_classes = [0, 8]  # must be two classes as we perform binary classification


def prepare_data(x, y, class_a=0, class_b=8):
    """
    This function convert the MNIST data to make it suitable for our binary classification
    setup.
    """
    # select items from the two selected classes
    mask = (y == class_a) + (
        y == class_b
    )  # mask to select only items from class_a or class_b
    x = x[mask]
    y = y[mask]
    x = x.astype("float32")
    y = y.astype("float32")
    # convert from range int[0,255] to float32[-1,1]
    x /= 255
    x = x.reshape((-1, 28, 28, 1))
    # change label to binary classification {-1,1}
    y[y == class_a] = 1.0
    y[y == class_b] = 0.0
    return x, y.reshape((-1, 1))


# now we load the dataset
(x_train, y_train_ord), (x_test, y_test_ord) = mnist.load_data()

# prepare the data
x_train, y_train = prepare_data(
    x_train, y_train_ord, selected_classes[0], selected_classes[1]
)
x_test, y_test = prepare_data(
    x_test, y_test_ord, selected_classes[0], selected_classes[1]
)

x_train = np.transpose(x_train,(0,3,1,2))
x_test = np.transpose(x_test,(0,3,1,2))

### Build lipschitz Model

Let's first explicit the paremeters of this experiment

In [5]:
# training parameters
epochs = 100
batch_size = 128

# network parameters
activation = GroupSort  # ReLU, MaxMin, GroupSort2

# loss parameters
min_margin = 1.0
alpha = 10.0


Now we can build the network.
Here the experiment is done with a MLP. But `Deel-lip` also provide state of the art 1-Lipschitz convolutions.

In [6]:
keras.utils.clear_session()
# helper function to build the 1-lipschitz MLP
model = Sequential(
    layers=[
        Input((1, 28, 28)),
        Flatten(),
        SpectralDense(32, activation=None, use_bias=True),
        GroupSort2(),
        SpectralDense(16, activation=None, use_bias=True),
        GroupSort2(),
        SpectralDense(1, activation=None, use_bias=False),
    ],
    name="lipModel",
)
model.summary()

I0000 00:00:1745509077.853231   25373 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 20413 MB memory:  -> device: 0, name: NVIDIA A10G, pci bus id: 0000:00:1e.0, compute capability: 8.6


Model: "lipModel"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spectral_dense (SpectralDense)  │ (None, 32)             │        50,241 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ group_sort2 (GroupSort2)        │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spectral_dense_1                │ (None, 16)             │         1,057 │
│ (SpectralDense)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ group_sort2_1 (GroupSort2)      │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spectral_dense_2                │ (None, 1)              │            34 │
│ (SpectralDense)                 │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 51,332 (200.52 KB)

 Trainable params: 25,664 (100.25 KB)

 Non-trainable params: 25,668 (100.27 KB)

In [7]:
model.compile(
    loss=HKR(
        alpha=alpha, min_margin=min_margin
    ),  # HKR stands for the hinge regularized KR loss
    metrics=[
        # KR,  # shows the KR term of the loss
        HingeMargin(min_margin=min_margin),  # shows the hinge term of the loss
    ],
    optimizer=Adam(learning_rate=0.001),
)

### Learn classification on MNIST

Now the model is build, we can learn the task.

In [8]:
model.fit(
    x=x_train,
    y=y_train,
    validation_data=(x_test, y_test),
    batch_size=batch_size,
    shuffle=True,
    epochs=epochs,
    verbose=1,
)

Epoch 1/100


I0000 00:00:1745509082.048358   25408 service.cc:148] XLA service 0x7fd0ec0d0530 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745509082.048402   25408 service.cc:156]   StreamExecutor device (0): NVIDIA A10G, Compute Capability 8.6
2025-04-24 17:38:02.102766: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1745509082.330342   25408 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-04-24 17:38:02.463078: W external/local_xla/xla/service/gpu/nvptx_compiler.cc:930] The NVIDIA driver's CUDA version is 12.4 which is older than the PTX compiler version 12.5.82. Because the driver is older than the PTX compiler version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.
2025-04-24 17:38:03.950913: I 

45/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - HingeMargin: 0.1082 - loss: -1.5420

I0000 00:00:1745509086.953021   25408 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


92/92 ━━━━━━━━━━━━━━━━━━━━ 9s 29ms/step - HingeMargin: 0.0784 - loss: -2.8071 - val_HingeMargin: 0.0268 - val_loss: -5.3800
Epoch 2/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - HingeMargin: 0.0317 - loss: -5.2870 - val_HingeMargin: 0.0196 - val_loss: -5.6263
Epoch 3/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - HingeMargin: 0.0225 - loss: -5.6110 - val_HingeMargin: 0.0212 - val_loss: -5.7211
Epoch 4/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - HingeMargin: 0.0233 - loss: -5.6774 - val_HingeMargin: 0.0195 - val_loss: -5.8136
Epoch 5/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - HingeMargin: 0.0207 - loss: -5.8119 - val_HingeMargin: 0.0204 - val_loss: -5.9034
Epoch 6/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - HingeMargin: 0.0196 - loss: -5.8845 - val_HingeMargin: 0.0211 - val_loss: -5.9616
Epoch 7/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - HingeMargin: 0.0191 - loss: -5.9291 - val_HingeMargin: 0.0236 - val_loss: -5.9627
Epoch 8/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - HingeMargin: 

In [9]:
vanilla_model = model.vanilla_export()

In [10]:
vanilla_model.compile(
    loss=HKR(
        alpha=alpha, min_margin=min_margin
    ),  # HKR stands for the hinge regularized KR loss
    metrics=[
        # KR,  # shows the KR term of the loss
        HingeMargin(min_margin=min_margin),  # shows the hinge term of the loss
    ],
    optimizer=Adam(learning_rate=0.001),
)

In [11]:
# def convert_reflected_two_logits_model(original_model):
#     """
#     Prend un modèle Keras qui sort un seul logit (z) et retourne un nouveau
#     modèle qui sort deux logits [-z, z].
#     """
#     inputs = original_model.inputs[0]
#     single_logit_output = original_model.outputs[0] # Sortie z, forme (batch, 1)

#     # Calculer -z en utilisant Keras Ops
#     neg_logit = keras.ops.negative(single_logit_output) # Forme (batch, 1)

#     # Concaténer [-z, z] en utilisant une couche Keras ou Keras Ops
#     # Utilisons keras.layers.Concatenate pour rester dans le style API Fonctionnelle
#     two_logits_output = keras.layers.Concatenate(axis=-1, name="reflected_two_logits")([neg_logit, single_logit_output]) # Forme (batch, 2)

#     # Créer le nouveau modèle
#     new_model = keras.Model(inputs=inputs, outputs=two_logits_output, name="reflected_two_logits_model")
#     return new_model

In [12]:
# outputs_2_vanilla_model.compile(
#     loss=MulticlassHKR(
#         alpha=alpha, min_margin=min_margin
#     ),  # HKR stands for the hinge regularized KR loss
#     metrics=[
#         # KR,  # shows the KR term of the loss
#         "accuracy",
#         HingeMargin(min_margin=min_margin),  # shows the hinge term of the loss
#     ],
#     optimizer=Adam(learning_rate=0.001),
# )

In [13]:
# layer = vanilla_model.layers[-1]
# new_dense = Dense(units=3, activation=None, use_bias=True)
# vanilla_model_bis = keras.models.Sequential(vanilla_model.layers[:-1] + [new_dense])

In [14]:
# new_dense(layer.input) # compile and erase weights

In [15]:
# w_temp = np.zeros((16,3), dtype = 'float32')
# b_temp = np.zeros((3,))
# b_temp[1:] = -10000

# w = layer.get_weights()[0] #(16,2)
# w_temp[:,:1] = w

In [16]:
# new_dense.set_weights([w_temp, b_temp])

In [17]:
# vanilla_model_bis.compile(
#         # decreasing alpha and increasing min_margin improve robustness (at the cost of accuracy)
#         # note also in the case of lipschitz networks, more robustness require more parameters.
#         loss=MulticlassHKR(alpha=100, min_margin=0.25),
#         optimizer=Adam(1e-4),
#         metrics=["accuracy", MulticlassKR()],)

In [18]:
# vanilla_model_bis.summary()

In [19]:
# vanilla_model_bis.evaluate(x_test, y_test)

In [20]:
# vanilla_model_bis.predict(x_test[:10])

In [21]:
# y_test[:10]

In [22]:
# def convert_reflected_two_logits_model2(original_model):
#     """
#     Prend un modèle Keras qui sort un seul logit (z) et retourne un nouveau
#     modèle qui sort deux logits [-z, z].
#     """
#     inputs = original_model.inputs
    
#     single_logit_output = original_model.outputs[0][:,:1] # Sortie z, forme (batch, 1)
#     print(single_logit_output.shape, single_logit_output)
#     # Calculer -z en utilisant Keras Ops
#     neg_logit = keras.ops.negative(single_logit_output) # Forme (batch, 1)

#     # Concaténer [-z, z] en utilisant une couche Keras ou Keras Ops
#     # Utilisons keras.layers.Concatenate pour rester dans le style API Fonctionnelle
#     two_logits_output = keras.layers.Concatenate(axis=-1, name="reflected_two_logits")([neg_logit, single_logit_output, original_model.outputs[0][:,1:3]]) # Forme (batch, 2)

#     # # Créer le nouveau modèle
#     new_model = keras.Model(inputs=inputs, outputs=[two_logits_output], name="reflected_two_logits_model")
#     return new_model

In [23]:
# outputs_2_vanilla_model = convert_reflected_two_logits_model2(vanilla_model_bis)

In [24]:
# y_test[:10]

In [25]:
# outputs_2_vanilla_model.predict(x_test[:10])

In [26]:
# outputs_2_vanilla_model.summary()

In [27]:
# outputs_2_vanilla_model.save("/home/aws_install/robustess_project/lip_models/demo3_FC_vanilla_MNIST08_channelfirst_False_disj_Neurons_single_output_converted_2.keras")
# vanilla_model.save("/home/aws_install/robustess_project/lip_models/demo3_FC_vanilla_MNIST08_channelfirst_False_disj_Neurons_single_output.keras")
# vanilla_model_bis.save("/home/aws_install/robustess_project/lip_models/demo3_FC_vanilla_MNIST08_channelfirst_False_disj_Neurons_single_output_converted_2_4_logits.keras")

# Test 4 outputs in 1


In [28]:
vanilla_model.summary()

Model: "lipModel"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spectral_dense (Dense)          │ (None, 32)             │        25,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ group_sort2 (GroupSort2)        │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spectral_dense_1 (Dense)        │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ group_sort2_1 (GroupSort2)      │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spectral_dense_2 (Dense)        │ (None, 1)              │            16 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,664 (100.25 KB)

 Trainable params: 25,664 (100.25 KB)

 Non-trainable params: 0 (0.00 B)

In [29]:
layer = vanilla_model.layers[-1]
new_dense = Dense(units=4, activation=None, use_bias=True)
vanilla_model_bis = keras.models.Sequential(vanilla_model.layers[:-1] + [new_dense])

In [30]:
new_dense(layer.input) # compile and erase weights

<KerasTensor shape=(None, 4), dtype=float32, sparse=False, ragged=False, name=keras_tensor_13>

In [31]:
w_temp = np.zeros((16,4), dtype = 'float32')
b_temp = np.zeros((4,))
b_temp[2:] = -10000

w = layer.get_weights()[0] 
w_temp[:,0:1] = -w
w_temp[:,1:2] = w

In [32]:
new_dense.set_weights([w_temp, b_temp])

In [61]:
vanilla_model_bis.compile(
        # decreasing alpha and increasing min_margin improve robustness (at the cost of accuracy)
        # note also in the case of lipschitz networks, more robustness require more parameters.
        loss=MulticlassHKR(alpha=100, min_margin=0.25),
        optimizer=Adam(1e-4),
        metrics=["accuracy", MulticlassKR()],)

# Test Accuracy

In [62]:
vanilla_model_bis.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (10, 784)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spectral_dense (Dense)          │ (10, 32)               │        25,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ group_sort2 (GroupSort2)        │ (10, 32)               │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spectral_dense_1 (Dense)        │ (10, 16)               │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ group_sort2_1 (GroupSort2)      │ (10, 16)               │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (10, 4)                │            68 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,716 (100.45 KB)

 Trainable params: 25,716 (100.45 KB)

 Non-trainable params: 0 (0.00 B)

In [63]:
vanilla_model_bis.predict(x_test[:10])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step


array([[-3.5752490e+00,  3.5752490e+00, -1.0000000e+04, -1.0000000e+04],
       [-3.2316532e+00,  3.2316532e+00, -1.0000000e+04, -1.0000000e+04],
       [-2.7814696e+00,  2.7814696e+00, -1.0000000e+04, -1.0000000e+04],
       [-4.9183712e+00,  4.9183712e+00, -1.0000000e+04, -1.0000000e+04],
       [-2.7872903e+00,  2.7872903e+00, -1.0000000e+04, -1.0000000e+04],
       [-1.1270375e+00,  1.1270375e+00, -1.0000000e+04, -1.0000000e+04],
       [ 1.6612896e+00, -1.6612896e+00, -1.0000000e+04, -1.0000000e+04],
       [-3.9001029e+00,  3.9001029e+00, -1.0000000e+04, -1.0000000e+04],
       [-4.5482106e+00,  4.5482106e+00, -1.0000000e+04, -1.0000000e+04],
       [ 3.0276570e+00, -3.0276570e+00, -1.0000000e+04, -1.0000000e+04]],
      dtype=float32)

train accuracy

In [64]:
np.count_nonzero(K.argmax(vanilla_model_bis(x_train), axis = 1) == y_train[:,0]) / x_train.shape[0]

0.9898929845422116

test accuracy

In [65]:
np.count_nonzero(K.argmax(vanilla_model_bis(x_test), axis = 1) == y_test[:,0]) / x_test.shape[0]

0.9902763561924258

In [66]:
#saving models
vanilla_model.save("/home/aws_install/robustess_project/lip_models/demo3_FC_vanilla_MNIST08_channelfirst_False_disj_Neurons_single_output.keras")
vanilla_model_bis.save("/home/aws_install/robustess_project/lip_models/demo3_FC_vanilla_MNIST08_channelfirst_False_disj_Neurons_single_output_converted_4logits.keras")

As we can see the model reach a very decent accuracy on this task.